In [1]:
import sys
sys.path.append("C:\\Users\\lauth\\OneDrive\\Desktop\\open_ai_assistant_v2")
from src.components.chains.chains import Assistant
import pandas as pd

assistant = Assistant(conversation_id="8af20de4-cd11-41da-9a9d-04762dd91322")
user_last_message = "Quiero una lista de los sistemas de medicion"

#### Generando un resumen de la conversación

In [3]:
assistant.add_user_message(message=user_last_message)
# assistant.add_ai_message(message="Aqui tienes la lista de los sistemas de Medicion: Medidor 1a, Medidor 2a, Medidor 3a")
# assistant.add_user_message(message="Hola")
# assistant.add_ai_message(message="Claro, ¿me puedes proporcionar un identificador del sistema de medición?")
# assistant.add_user_message(message="me dices cuales son los ids que existen?")
# assistant.add_user_message(message="el Tag es F980-40")
# assistant.add_ai_message(message="Por supuesto, ¿que tipo de información requieres?")
# assistant.add_user_message(message="Me gustaria saber el promedio de la presion en las fechas de enero y febrero")
# assistant.add_user_message(message="Necesito una lista de los sistemas de medicion de gas")
summary = assistant.get_memory_summary()
print(summary)


        The human is asking for a list of measurement systems.


#### Generamos cual es el requerimiento del usuario del resumen

In [4]:
user_request = assistant.get_request(summary)
print(user_request["request"])

The human is requesting a list of measurement systems.


#### Clasificamos el requerimiento

In [5]:
request_type = assistant.classify_request(user_request["request"])
print(request_type)

{'analysis': 'This is a complete request, the human is asking for a list of measurement systems and I can not answer with the context', 'response': 'complex-complete'}


In [6]:
def get_response(
    request_type: str,
    user_request: str,
    coversation_summary: str,
    user_last_message: str,
):
    if request_type == "simple":
        res = assistant.greeting_response(coversation_summary, user_last_message)
        return res
    elif request_type == "complex-incomplete":
        # keywords = assistant.get_keywords_from_requirement(user_request)
        # tables = assistant.get_table_names(keywords)
        # print(keywords)
        # print(tables)
        print("No esta completo")
        return
    elif request_type == "complex-complete":
        df = assistant.get_answer_from_db(user_request)
        res = assistant.get_complete_response(
            user_request=user_request, context_dataframe=df
        )
        return res
    else:
        raise ValueError(f"Not support request type: {request_type}")

In [7]:
request = user_request["request"]
request_type = request_type["response"]
summary = summary

res = get_response(request_type, request, summary, user_last_message)

C:\Users\lauth\OneDrive\Desktop\open_ai_assistant_v2\src\tools\vanna\vanna_tool.py:40: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(sql, self.conn)


In [8]:
import pandas as pd

final_response = res["response"]
final_df : pd.DataFrame = res["dataframe"]

print(final_response)
final_df.head(None)


The DataFrame contains three columns: Id, Nombre, and Tag. The Id column is a unique identifier for each row in the DataFrame. The Nombre column contains a string of characters that represents a measurement system. The Tag column also contains a string of characters that represents a measurement system, but it is a more specific identifier than the Nombre column. The DataFrame contains 10 examples of measurement systems, but it is likely that there are many more measurement systems in the complete DataFrame.


,Id,Nombre,Tag
0,1,EMED-3138.11-128,EMED-3138.11-128
1,2,EMED-3138.12-050,EMED-3138.12-050
2,3,EMED-3138.12-052,EMED-3138.12-052
3,4,EMED-3138.12-065,EMED-3138.12-065
4,5,EMED-3138.12-105,EMED-3138.12-105
...,...,...,...
145,146,TANQ-3141.03-218_OP,TANQ-3141.03-218_OP
146,147,TANQ-3141.03-206_OP,TANQ-3141.03-206_OP
147,148,TANQ-3138.04-202_OP,TANQ-3138.04-202_OP
148,149,TI-TQ-3138.11-002(Meio),TI-TQ-3138.11-002(Meio)


In [7]:
assistant.add_ai_message(message=res)

In [2]:
import sys
sys.path.append("C:\\Users\\lauth\\OneDrive\\Desktop\\open_ai_assistant_v2")
from src.components.chains.chains import Assistant
import pandas as pd

assistant = Assistant(conversation_id="8af20de4-cd11-41da-9a9d-04762dd91322")

def get_response(
    request_type: str,
    user_request: str,
    coversation_summary: str,
    user_last_message: str,
):
    response = {
        "answer" : "",
        "dataframe" : []
    }
    
    if request_type == "simple":
        response["answer"] = assistant.greeting_response(coversation_summary, user_last_message)
    elif request_type == "complex-incomplete":
        # keywords = assistant.get_keywords_from_requirement(user_request)
        # tables = assistant.get_table_names(keywords)
        # print(keywords)
        # print(tables)
        response["answer"] = "No esta completo"
    elif request_type == "complex-complete":
        df = assistant.get_answer_from_db(user_request)
        res = assistant.get_complete_response(
            user_request=user_request, context_dataframe=df
        )
        response["answer"] = res
        response["dataframe"].append(df)
    else:
        raise ValueError(f"Not support request type: {request_type}")
    
    return response

def chat(user_message : str):
    # Generamos el resumen
    assistant.add_user_message(message=user_message)
    summary = assistant.get_memory_summary()
    
    # Generamos el requerimiento
    user_request = assistant.get_request(summary)
    
    # Clasificamos el requerimiento
    request_type = assistant.classify_request(user_request["request"])
    
    # Obtenemos una respuesta
    first_response = get_response(request_type["response"], user_request["request"], summary, user_message)
    
    # Procesamos la respuesta para entregar una respuesta final
    final_response = assistant.process_final_response(user_message, first_response["answer"])
    
    # Actualizando el chat
    assistant.add_ai_message(message=final_response["final_answer"])
    
    if len(first_response["dataframe"]) > 0:
        return final_response["final_answer"], first_response["dataframe"]
    else:
        return final_response["final_answer"]

In [6]:
user_last_message = "quiero una lista de los sistemas de medicion"
res = chat(user_last_message)

C:\Users\lauth\OneDrive\Desktop\open_ai_assistant_v2\src\tools\vanna\vanna_tool.py:40: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(sql, self.conn)



"final_answer": "The DataFrame contiene tres columnas: Id, Nombre, y Tag. La columna Id es un identificador único para cada fila en el DataFrame. La columna Nombre contiene una cadena de caracteres que representa un sistema de medición. La columna Tag también contiene una cadena de caracteres que representa un sistema de medición, pero es un formato diferente al de la columna Nombre. El DataFrame contiene 10 ejemplos de sistemas de medición, pero probablemente hay más sistemas de medición en el DataFrame completo."


In [7]:
print(res[0])
print(res[1][0])

The DataFrame contiene tres columnas: Id, Nombre, y Tag. La columna Id es un identificador único para cada fila en el DataFrame. La columna Nombre contiene una cadena de caracteres que representa un sistema de medición. La columna Tag también contiene una cadena de caracteres que representa un sistema de medición, pero es un formato diferente al de la columna Nombre. El DataFrame contiene 10 ejemplos de sistemas de medición, pero probablemente hay más sistemas de medición en el DataFrame completo.
      Id                   Nombre                      Tag
0      1         EMED-3138.11-128         EMED-3138.11-128
1      2         EMED-3138.12-050         EMED-3138.12-050
2      3         EMED-3138.12-052         EMED-3138.12-052
3      4         EMED-3138.12-065         EMED-3138.12-065
4      5         EMED-3138.12-105         EMED-3138.12-105
..   ...                      ...                      ...
145  146      TANQ-3141.03-218_OP      TANQ-3141.03-218_OP
146  147      TANQ-3141.0

In [9]:
assistant.get_memory_summary()

'\n        The human is asking AI for its name and AI responds with M-Assistant and asks how it can help. The human then asks for a list of measurement systems and AI explains that the DataFrame contains three columns: Id, Nombre, and Tag. The Id column is a unique identifier for each row in the DataFrame. The Nombre column contains a string that represents a measurement system. The Tag column also contains a string that represents a measurement system, but in a different format than the Nombre column. The DataFrame contains 10 examples of measurement systems, but there are likely more in the complete DataFrame.'